![logo](https://awl.co.jp/wp-content/themes/awl/img/img-header-logo.png)

# AI Music Generator Project (Melody_RNN)

In [ ]:
from ipywidgets import widgets

config= widgets.RadioButtons(
    options=["basic_rnn","mono_rnn", "lookback_rnn", "attention_rnn"],
    description='Config:',
    disabled=False
)

training_steps = widgets.IntSlider(
    value=500,
    min=0,
    max=4000,
    step=10,
    description='Training Steps:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

train_validation_percent = widgets.FloatSlider(
    value=0.9,
    min=0.1,
    max=1,
    step=0.05,
    description='Train/Val Ratio',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)


batch_size= widgets.RadioButtons(
    options=['64', '128', '264'],
    description='Batch Size:',
    disabled=False
)

output_folder = widgets.Text(
    value='run1',
    placeholder='output folder name',
    description='output folder name:',
    disabled=False
)

num_midis = widgets.IntSlider(
    value=2,
    min=1,
    max=10,
    step=1,
    description='Num of Midis:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

Number_of_Steps = widgets.IntSlider(
    value=128,
    min=64,
    max=1000,
    step=64,
    description='Number_of_Steps:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

primer_pitches = widgets.IntSlider(
    value=64,
    min=0,
    max=127,
    step=1,
    description='primer_pitch:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

primer_pitch = '['+str(primer_pitches.value) +']'

display(config)
display(training_steps)
display(train_validation_percent)
display(batch_size)
display(output_folder)
display(num_midis)
display(Number_of_Steps)
display(primer_pitches)


In [ ]:
import os

drive_dir = "/root/magenta-data"

training_dir = os.path.join(drive_dir,"midi-data/training-set")
generated_midi_dir = os.path.join(drive_dir,"midi-data/generated_data/melody_rnn")

note_dir = os.path.join(drive_dir,"tmp")
seq_dir = os.path.join(drive_dir,"tmp/melody_rnn/sequence_examples")

#congfig
os.environ['CONFIG']=str(config.value)


os.environ['TRAIN_SET_DIR'] = str(os.path.join(drive_dir,"midi-data/training-set"))
os.environ['NOTESEQ_FILE'] = str(os.path.join(drive_dir,"tmp/notesequences.tfrecord"))
os.environ['SEQ_DIR'] = str(os.path.join(drive_dir,"tmp/melody_rnn/sequence_examples"))
os.environ['SEQ_FILE']= str(os.path.join(drive_dir,"tmp/melody_rnn/sequence_examples/eval_melodies.tfrecord"))
os.environ['RUN_DIR'] = str(os.path.join(drive_dir,"tmp/melody_rnn/logdir/run1"))
os.environ['OUTPUT_DIR'] = str(os.path.join(drive_dir,"midi-data/generated_data/melody_rnn"))


os.environ['TRAINING_STEPS']=str(training_steps.value)
os.environ['TRAIN_VAL_RATIO']= str(train_validation_percent.value)
os.environ['BATCH_SIZE']=str(batch_size.value)

#hparams
hparams='batch_size='+str(batch_size.value) +",rnn_layer_sizes="+str("[64,64]")
os.environ['HPARAMS']=hparams

# generation
generated_midi_dir_temp=os.path.join(generated_midi_dir, output_folder.value)

os.environ['OUTPUT_DIR']= generated_midi_dir_temp
os.environ['MIDI_OUTPUT']=str(num_midis.value)
os.environ['GEN_STEPS']=str(Number_of_Steps.value)
os.environ['PRIMER_PITCH']=primer_pitch




In [ ]:
!convert_dir_to_note_sequences \
--input_dir="$TRAIN_SET_DIR" \
--output_file="$NOTESEQ_FILE"

In [ ]:
!melody_rnn_create_dataset \
--config=$CONFIG \
--input="$NOTESEQ_FILE" \
--output_dir="$SEQ_DIR" \
--eval_ratio=$TRAIN_VAL_RATIO 

In [ ]:
!melody_rnn_train \
--config=$CONFIG \
--run_dir="$RUN_DIR" \
--sequence_example_file="$SEQ_FILE" \
--hparams=$HPARAMS \
--num_training_steps=$TRAINING_STEPS

In [ ]:
!melody_rnn_generate \
--config=$CONFIG \
--run_dir="$RUN_DIR" \
--output_dir="$OUTPUT_DIR" \
--num_outputs=$MIDI_OUTPUT \
--num_steps=$GEN_STEPS \
--hparams=$HPARAMS \
--primer_melody="$PRIMER_PITCH"